In [5]:
import pandas as pd
import numpy as np
import re
import os

In [6]:
dl=[]
fl=os.listdir("wthor_data")
for filename in fl:
    filepath="wthor_data"+"\\"+filename
    dl.append(pd.read_csv(filepath))

df=pd.concat(dl)
df=df.drop(index=df[df["transcript"].str.contains('transcript')].index)
df

,tournamentId,tournamentName,blackPlayerId,blackPlayerName,whitePlayerId,whitePlayerName,blackScore,blackTheoreticalScore,transcript
0,116,Australian National,1233,Hubbard Paul,1728,Vinar Matt,11,14,f5d6c5f4e3c6d3f6e6d7g3c4g5c3f7d2e7f2c8f3c7h3e2...
1,116,Australian National,1728,Vinar Matt,1233,Hubbard Paul,16,28,f5d6c4d3c3f4f6b5b4f3e6e7f7c5e3a3c7d7b6g6g5c2a6...
2,116,Australian National,1728,Vinar Matt,150,Ortiz Georges,61,55,f5f6e6f4g5e7e3g6d6c4f7h5h6h7d7d8c6e8c5h4f3b6c8...
3,116,Australian National,1732,Girardo Robert,1728,Vinar Matt,8,13,f5d6c4d3c5f4e3f3d7c3d2g6b3b4b5d1e2c2c1e1g4g5f6...
4,116,Australian National,1728,Vinar Matt,1731,Cairns Fleur,52,46,f5f6e6f4g5c6g6h7g4h6h4e7f3e3d6f2d8d7h5h3g3f8c8...
...,...,...,...,...,...,...,...,...,...
2167,164,Zonhoven Open,1708,Hobo Roel,1935,Vaes Dimitri,41,45,f5d6c3d3c4f4f6f3g4g5e3h3e6c5g3d7h5e2h6d2d1e1f1...
2168,164,Zonhoven Open,1708,Hobo Roel,1971,Schotte Tom,46,36,f5f6e6f4e3c5c4e7c6e2f3d3c3d2e1c1f2c2d1f1g6d6f7...
2169,164,Zonhoven Open,1708,Hobo Roel,2544,Helmes Jiska,18,40,f5d6c3d3c4f4e3g5c6c5d7e7e6b4b6e2d2b3c2c1a4a5a6...
2170,164,Zonhoven Open,1935,Vaes Dimitri,1708,Hobo Roel,46,39,f5d6c3d3c4f4c5b4b5b3d7c6f3f6f7e6c7e7e8b6e3c8a6...


In [7]:

# 正規表現を使って2文字ずつ切り出す
transcripts_raw = df["transcript"].str.extractall('(..)')

# Indexを再構成して、1行1手の表にする
transcripts_df = transcripts_raw.reset_index().rename(columns={"level_0":"tournamentId" , "match":"move_no", 0:"move_str"})

# 列の値を数字に変換するdictonaryを作る
def left_build_conv_table():
    left_table = ["a","b","c","d","e","f","g","h"]
    left_conv_table = {}
    n = 1

    for t in left_table:
        left_conv_table[t] = n
        n = n + 1

    return left_conv_table

left_conv_table = left_build_conv_table()

# dictionaryを使って列の値を数字に変換する
def left_convert_colmn_str(col_str):
    return left_conv_table[col_str]

# 1手を数値に変換する
def convert_move(v):
    l = left_convert_colmn_str(v[:1]) # 列の値を変換する
    r = int(v[1:]) # 行の値を変換する
    return np.array([l - 1, r - 1], dtype='int8')

transcripts_df["move"] = transcripts_df.apply(lambda x: convert_move(x["move_str"]), axis=1)

In [8]:
# 盤面の中にあるかどうかを確認する
def is_in_board(cur):
  return cur >= 0 and cur <= 7

# ある方向(direction）に対して石を置き、可能なら敵の石を反転させる
def put_for_one_move(board_a, board_b, move_row, move_col, direction):
  board_a[move_row][move_col] = 1

  tmp_a = board_a.copy()
  tmp_b = board_b.copy()
  cur_row = move_row
  cur_col = move_col

  cur_row += direction[0]
  cur_col += direction[1]
  reverse_cnt = 0
  while is_in_board(cur_row) and is_in_board(cur_col):
    if tmp_b[cur_row][cur_col] == 1: # 反転させる
      tmp_a[cur_row][cur_col] = 1
      tmp_b[cur_row][cur_col] = 0
      cur_row += direction[0]
      cur_col += direction[1]
      reverse_cnt += 1
    elif tmp_a[cur_row][cur_col] == 1:
      return tmp_a, tmp_b, reverse_cnt
    else:
      return board_a, board_b, reverse_cnt
  return board_a, board_b, reverse_cnt

# 方向の定義（配列の要素は←、↖、↑、➚、→、➘、↓、↙に対応している）
directions = [[-1,0],[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1]]

# ある位置に石を置く。すべての方向に対して可能なら敵の石を反転させる
def put(board_a, board_b ,move_row, move_col):
  tmp_a = board_a.copy()
  tmp_b = board_b.copy()
  global directions
  reverse_cnt_amount = 0
  for d in directions:
    board_a ,board_b, reverse_cnt = put_for_one_move(board_a, board_b, move_row, move_col, d)
    reverse_cnt_amount += reverse_cnt

  return board_a , board_b, reverse_cnt_amount

# 盤面の位置に石がないことを確認する
def is_none_state(board_a, board_b, cur_row, cur_col):
  return board_a[cur_row][cur_col] == 0 and board_b[cur_row][cur_col] == 0

# 盤面に石が置けるかを確認する（ルールでは敵の石を反転できるような位置にしか石を置けない）  
def can_put(board_a, board_b, cur_row, cur_col):
  copy_board_a = board_a.copy()
  copy_board_b = board_b.copy()
  _,  _, reverse_cnt_amount = put(copy_board_a, copy_board_b, cur_row, cur_col)
  return reverse_cnt_amount > 0

# パスする必要のある盤面かを確認する
def is_pass(is_black_turn, board_black, board_white):
  if is_black_turn:
    own = board_black
    opponent = board_white
  else:
    own = board_white
    opponent = board_black
  for cur_row in range(8):
      for cur_col in range(8):
        if is_none_state(own, opponent, cur_row, cur_col) and can_put(own, opponent, cur_row, cur_col):
          return False
  return True

# 変数の初期化
b_tournamentId = -1 # トーナメント番号
board_black = [] # 黒にとっての盤面の状態（１試合保存用）
board_white = [] # 白にとっての盤面の状態（１試合保存用）
boards_black = [] # 黒にとっての盤面の状態（全トーナメント保存用）
boards_white = [] # 白にとっての盤面の状態（全トーナメント保存用）
moves_black = [] # 黒の打ち手（全トーナメント保存用）
moves_white = [] # 白の打ち手（全トーナメント保存用）
is_black_turn = True # True = 黒の番、 False = 白の番
# ターン（黒の番 or 白の番）を切り変える
def switch_turn(is_black_turn):
  return is_black_turn == False # ターンを切り替え

# 棋譜のデータを１つ読み、学習用データを作成する関数
def process_tournament(df):
  global is_black_turn
  global b_tournamentId
  global boards_white
  global boards_black
  global board_white
  global board_black
  global moves_white
  global moves_black
  if df["tournamentId"] != b_tournamentId:
    # トーナメントが切り替わったら盤面を初期状態にする
    b_tournamentId = df["tournamentId"]
    board_black = np.zeros(shape=(8,8), dtype='int8')
    board_black[3][4] = 1
    board_black[4][3] = 1
    board_white = np.zeros(shape=(8,8), dtype='int8')
    board_white[3][3] = 1
    board_white[4][4] = 1
    is_black_turn = True
  else:
    # ターンを切り替える
    is_black_turn = switch_turn(is_black_turn)
    if is_pass(is_black_turn, board_black, board_white): # パスすべき状態か確認する
      is_black_turn = switch_turn(is_black_turn) #パスすべき状態の場合はターンを切り替える

  # 黒の番なら黒の盤面の状態を保存する、白の番なら白の盤面の状態を保存する
  if is_black_turn:
    boards_black.append(np.array([board_black.copy(), board_white.copy()], dtype='int8'))
  else:
    boards_white.append(np.array([board_white.copy(), board_black.copy()], dtype='int8'))

  # 打ち手を取得する
  move = df["move"]
  #move_one_hot = np.zeros(shape=(8,8), dtype='int8')
  #move_one_hot[move[1]][move[0]] = 1
  m=8*move[1]+(move[0]+1)

  # 黒の番なら自分→敵の配列の並びをを黒→白にして打ち手をセットする。白の番なら白→黒の順にして打ち手をセットする
  if is_black_turn:
    #moves_black.append(move_one_hot)
    moves_black.append(m)
    board_black, board_white, _ = put(board_black, board_white, move[1], move[0])
  else:
    #moves_white.append(move_one_hot)
    moves_white.append(m)
    board_white, board_black, _ = put(board_white, board_black, move[1], move[0])


# 棋譜データを学習データに展開する
transcripts_df.apply(lambda x: process_tournament(x), axis= 1)

0          None
1          None
2          None
3          None
4          None
           ... 
3140631    None
3140632    None
3140633    None
3140634    None
3140635    None
Length: 3140636, dtype: object

In [9]:
boards_black

[array([[[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],
 
        [[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]]], dtype=int8),
 array([[[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],
 
        [[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],

In [10]:
x_train = np.concatenate([boards_black, boards_white])
y_train = np.concatenate([moves_black, moves_white])
# 教師データは8x8の2次元データになっているので、64要素の1次元データにreshapeする
#y_train_reshape = y_train.reshape(-1,64)

In [17]:
x_train=np.reshape(x_train,[-1,128])
x_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int8)

In [1]:
len(y_train)

NameError: name 'y_train' is not defined

In [16]:
np.save("train_x",x_train)
np.save("train_y",y_train)